In [ ]:
!pip install radgraph tqdm pandas Pillow

^C


ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\clint\\miniconda3\\Lib\\site-packages\\spacy\\lang\\fa\\punctuation.py'
Consider using the `--user` option or check the permissions.



  Using cached radgraph-0.1.2-py3-none-any.whl
  Using cached spacy-3.8.7-cp313-cp313-win_amd64.whl.metadata (28 kB)
Using cached spacy-3.8.7-cp313-cp313-win_amd64.whl (13.9 MB)


### Proccess Collected Images to Match Chest Xray Views also check the validity of collected studies

In [ ]:
import os
import shutil
import pandas as pd


MIMIC_FOLDER = 'MIMIC_Dataset'
METADATA_CSV = 'mimic-cxr-2.0.0-metadata.csv'
OUTPUT_FOLDER = 'Processed_MIMIC'
BAD_FOLDER = 'invalid_studies1'

FRONTAL_VIEWS = {'PA', 'AP', 'AP AXIAL', 'PA LLD', 'AP LLD', 'AP RLD', 'PA RLD', 'LPO', 'RAO', 'LAO'}
LATERAL_VIEWS = {'LATERAL', 'LL', 'XTABLE LATERAL', 'SWIMMERS'}

def map_view(view_str):
    if pd.isna(view_str):
        return None
    view_str = view_str.strip().upper()
    if view_str in FRONTAL_VIEWS:
        return '0'
    elif view_str in LATERAL_VIEWS:
        return '1'
    return None

def infer_view_from_filename(filename):
    for view in FRONTAL_VIEWS:
        if view in filename.upper():
            return '0'
    for view in LATERAL_VIEWS:
        if view in filename.upper():
            return '1'
    return None


metadata = pd.read_csv(METADATA_CSV)
metadata['view_label'] = metadata['ViewPosition'].apply(map_view)
metadata.dropna(subset=['view_label'], inplace=True)


study_to_dicom = {}
for _, row in metadata.iterrows():
    study_id = str(row['study_id'])
    if study_id not in study_to_dicom:
        study_to_dicom[study_id] = []
    study_to_dicom[study_id].append((row['dicom_id'], row['view_label']))


os.makedirs(OUTPUT_FOLDER, exist_ok=True)
os.makedirs(BAD_FOLDER, exist_ok=True)

for subject_id in os.listdir(MIMIC_FOLDER):
    subject_path = os.path.join(MIMIC_FOLDER, subject_id)
    if not os.path.isdir(subject_path):
        continue

    study_view_map = {}

    for file_name in os.listdir(subject_path):
        if not file_name.lower().endswith(('.jpeg', '.jpg', '.png')):
            continue

        image_path = os.path.join(subject_path, file_name)

     
        matched_study = None
        matched_label = None
        for study_id, dicom_entries in study_to_dicom.items():
            for dicom_id, label in dicom_entries:
                if dicom_id in file_name:
                    matched_study = study_id
                    matched_label = label
                    break
            if matched_study:
                break

        if matched_study:
            if matched_study not in study_view_map:
                study_view_map[matched_study] = {'0': None, '1': None}
            if matched_label in study_view_map[matched_study] and study_view_map[matched_study][matched_label] is None:
                study_view_map[matched_study][matched_label] = (image_path, matched_label)
            continue

      
        inferred_label = infer_view_from_filename(file_name)
        if inferred_label is not None:
            out_study_path = os.path.join(OUTPUT_FOLDER, subject_id)
            os.makedirs(out_study_path, exist_ok=True)
            dst = os.path.join(out_study_path, f"{inferred_label}.jpeg")
            try:
                shutil.copy(image_path, dst)
              
            except Exception as e:
                print(f"Error copying {image_path} → {dst}: {e}")
            continue

     
        bad_path = os.path.join(BAD_FOLDER, subject_id)
        os.makedirs(bad_path, exist_ok=True)
        shutil.copy(image_path, os.path.join(bad_path, file_name))
       

  
    for study_id, view_dict in study_view_map.items():
        if not view_dict['0'] or not view_dict['1']:
            bad_path = os.path.join(BAD_FOLDER, subject_id)
            os.makedirs(bad_path, exist_ok=True)
            for v in view_dict.values():
                if v:
                    shutil.copy(v[0], os.path.join(bad_path, os.path.basename(v[0])))
           
            continue

        out_study_path = os.path.join(OUTPUT_FOLDER, subject_id)
        os.makedirs(out_study_path, exist_ok=True)
        for original_file, label in view_dict.values():
            dst = os.path.join(out_study_path, f"{label}.jpeg")
            try:
                shutil.copy(original_file, dst)
            except Exception as e:
                print(f"Error copying {original_file} → {dst}: {e}")
print("Processing Completed")

#### Extract Valid Study Ids from Processed Images Folder (For Cleaning the text Using Radgraph)

In [ ]:
import os
import pandas as pd
from pathlib import Path

base_dir = Path("Processed_MIMIC")

valid_study_ids = []

for study_folder in base_dir.iterdir():
    if study_folder.is_dir():
        image0 = study_folder / "0.jpeg"
        image1 = study_folder / "1.jpeg"
        
        if image0.exists() and image1.exists():
            valid_study_ids.append(study_folder.name)
       

df = pd.DataFrame(valid_study_ids, columns=["study_id"])
df.to_csv("study_ids.csv", index=False)

print(f"Saved {len(valid_study_ids)} valid study IDs to valid_study_ids.csv")


Saved 34122 valid study IDs to valid_study_ids.csv


#### 

In [ ]:
import zipfile
import json
import os
import re
import csv
from tqdm import tqdm
from pathlib import Path
import pandas as pd
import transformers


ZIP_PATH = 'mimic-cxr-reports.zip'
IMAGE_ROOT = 'study_ids.csv'
OUTPUT_JSON = "radgraph_processed_output.json"
valid_id_path= "valid_pa_lateral_studies.csv"

def load_valid_ids(csv_path):
    valid_ids = set()
    with open(csv_path, newline='') as f:
        reader = csv.DictReader(f)
        print(reader)
        for row in reader:
            subject_id = row["subject_id"].strip()
            study_id = row["study_id"].strip()
            valid_ids.add((subject_id, study_id))
    return valid_ids


def extract_reports(zip_path):
    reports = {}
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        txt_files = [f for f in zip_ref.namelist() if f.endswith('.txt')]
        for file_path in tqdm(txt_files, desc="Extracting reports"):
            try:
                parts = file_path.split('/')
                if len(parts) >= 4 and parts[-1].startswith('s'):
                    with zip_ref.open(file_path) as f:
                        content = f.read().decode('utf-8')
                        subject_id = parts[-2][1:] 
                        study_id = parts[-1][1:-4]  
                        reports[(subject_id, study_id)] = content
            except Exception as e:
                print(f"Error reading {file_path}: {e}")
    return reports


def extract_findings_impression(text):
    text = text.upper()
    findings_match = re.search(
        r"(FINDINGS|OBSERVATION|DESCRIPTION):\s*(.*?)(IMPRESSION|CONCLUSION|DIAGNOSIS|$)",
        text,
        re.DOTALL| re.IGNORECASE
    )
    impression_match = re.search(
        r"(IMPRESSION|CONCLUSION|DIAGNOSIS):\s*(.*)",
        text,
        re.DOTALL| re.IGNORECASE
    )

    findings = findings_match.group(2).strip() if findings_match else ""
    impression = impression_match.group(2).strip() if impression_match else ""
    return findings, impression


def process_reports(zip_path, valid_ids_path, image_root, output_json):
    valid_ids = load_valid_ids(valid_ids_path)
    reports = extract_reports(zip_path)
    result = {}


    study_pd = pd.read_csv(image_root)
    study_dirs = study_pd['study_id'].tolist()

    for study_id in tqdm(study_dirs, desc="Processing study folders"):

        subject_id = next((sid for sid, sidy in valid_ids if int(sidy) == study_id), None)
        if subject_id is None:
            continue
        report_text = reports.get((subject_id, str(study_id)))
        if report_text is None:
            continue

        findings, impression = extract_findings_impression(report_text)
        if not findings or not impression:
            continue

        base_prefix = "Processed_MIMIC"
        image_paths = [str(study_id)+"/0.jpeg", str(study_id)+"/1.jpeg"]
        result[study_id] = {
            "subject_id": subject_id,
            "findings": findings.replace("\n", " ").strip(),
            "impression": impression.replace("\n", " ").strip(),
            "image_paths": image_paths
        }

    return result

result = process_reports(ZIP_PATH,  valid_id_path,IMAGE_ROOT,OUTPUT_JSON)

Processing study folders: 100%|██████████████████████████████████████████████████| 34122/34122 [15:05<00:00, 37.67it/s]


In [5]:
OUTPUT_JSON_drive = "radgraph_processed_output.json"
with open(OUTPUT_JSON_drive, "w") as f:
        json.dump(result, f, indent=2)


#### Resize center Crop Imges  to 224x224

In [ ]:
import os
import json
import cv2
import numpy as np
from tqdm import tqdm


json_path = "radgraph_processed_output.json" 
base_prefix = "Processed_MIMIC"          
save_prefix = "MIMIC_Dataset336_stretched"  
output_size = 336                         

os.makedirs(save_prefix, exist_ok=True)



def crop_and_stretch(img, final_size=336):
    """
    Crop black borders and stretch to final_size x final_size.
    """
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY)
    coords = cv2.findNonZero(thresh)

    if coords is None:
        return cv2.resize(img, (final_size, final_size))  

    x, y, w, h = cv2.boundingRect(coords)
    cropped = img[y:y+h, x:x+w]
    stretched = cv2.resize(cropped, (final_size, final_size), interpolation=cv2.INTER_LANCZOS4)
    return stretched

def resize_image(input_path, output_path, size=336):
    img = cv2.imread(input_path)
    if img is None:
        raise FileNotFoundError(f"Failed to load image: {input_path}")

    processed = crop_and_stretch(img, final_size=size)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    cv2.imwrite(output_path, processed)



def resize_images_from_json(json_path, base_prefix, save_prefix, size=336):
    with open(json_path, "r") as f:
        data = json.load(f)

    for study_id, info in tqdm(data.items(), desc="Processing studies"):
        image_paths = info.get("image_paths", [])
        if len(image_paths) != 2:
            continue  # Skip non-dual-view entries

        for rel_path in image_paths:
            rel_path = rel_path.strip("/")
            full_input = os.path.join(base_prefix, rel_path)
            full_output = os.path.join(save_prefix, rel_path)

            try:
                resize_image(full_input, full_output, size=size)
            except Exception as e:
                print(f"Failed to process {full_input}: {e}")



if __name__ == "__main__":
    resize_images_from_json(json_path, base_prefix, save_prefix, size=output_size)


Processing studies: 100%|██████████████████████████████████████████████████████| 32734/32734 [9:09:13<00:00,  1.01s/it]


In [ ]:
import os
import json
import cv2
import numpy as np
from tqdm import tqdm

# ----------------------
# CONFIG
# ----------------------
json_path = "radgraph_processed_output.json" 
base_prefix = "Processed_MIMIC"          
save_prefix = "MIMIC_Dataset224_clean"  
output_size = 224                      

os.makedirs(save_prefix, exist_ok=True)


def crop_and_resize_with_padding(img, final_size=224):
    """
    Remove black borders and resize to square with padding (no stretching).
    """
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY)
    coords = cv2.findNonZero(thresh)

    if coords is None:
        return cv2.resize(img, (final_size, final_size))

    x, y, w, h = cv2.boundingRect(coords)
    cropped = img[y:y+h, x:x+w]


    h, w = cropped.shape[:2]
    scale = final_size / max(h, w)
    new_w, new_h = int(w * scale), int(h * scale)
    resized = cv2.resize(cropped, (new_w, new_h), interpolation=cv2.INTER_LANCZOS4)

    top_pad = (final_size - new_h) // 2
    bottom_pad = final_size - new_h - top_pad
    left_pad = (final_size - new_w) // 2
    right_pad = final_size - new_w - left_pad

    padded = cv2.copyMakeBorder(resized, top_pad, bottom_pad, left_pad, right_pad,
                                borderType=cv2.BORDER_CONSTANT, value=[0, 0, 0])
    return padded


def resize_image(input_path, output_path, size=224):
    img = cv2.imread(input_path)
    if img is None:
        raise FileNotFoundError(f"Failed to load image: {input_path}")

    processed = crop_and_resize_with_padding(img, final_size=size)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    cv2.imwrite(output_path, processed)

def resize_images_from_json(json_path, base_prefix, save_prefix, size=224):
    with open(json_path, "r") as f:
        data = json.load(f)

    for study_id, info in tqdm(data.items(), desc="Processing studies"):
        image_paths = info.get("image_paths", [])
        if len(image_paths) != 2:
            continue  # Skip non-dual-view entries

        for rel_path in image_paths:
            rel_path = rel_path.strip("/")
            full_input = os.path.join(base_prefix, rel_path)
            full_output = os.path.join(save_prefix, rel_path)

            try:
                resize_image(full_input, full_output, size=size)
            except Exception as e:
                print(f"Failed to process {full_input}: {e}")


if __name__ == "__main__":
    resize_images_from_json(json_path, base_prefix, save_prefix, size=output_size)


Processing studies: 100%|██████████████████████████████████████████████████████| 32734/32734 [8:02:43<00:00,  1.13it/s]
